### Import MobileNet model given in file `mn_model.py`

In [1]:
### Import MobileNet model given in file `mn_model.py`

from mn_model import mn_model

ModuleNotFoundError: No module named 'mn_model'

In [2]:
#### Import the BatchGenerator and SSDLoss functions as well, used in MobileNet model

from face_generator import BatchGenerator
from keras_ssd_loss import SSDLoss
from ssd_box_encode_decode_utils import SSDBoxEncoder, decode_y, decode_y2

In [3]:

import warnings
warnings.filterwarnings("ignore")

from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, LearningRateScheduler
from keras.callbacks import Callback
from keras import backend as K 
from keras.models import load_model
from math import ceil 
import numpy as np 
from termcolor import colored

from mn_model import mn_model
from face_generator import BatchGenerator
from keras_ssd_loss import SSDLoss
from ssd_box_encode_decode_utils import SSDBoxEncoder, decode_y, decode_y2

# training parameters
from keras import backend as K
import scipy.misc as sm


In [4]:
img_height =512
img_width = 512

img_channels = 3

n_classes =2 
class_names = ["background","face"]

scales = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # anchorboxes for coco dataset
aspect_ratios = [[0.5, 1.0, 2.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [0.5, 1.0, 2.0],
                 [0.5, 1.0, 2.0]] # The anchor box aspect ratios used in the original SSD300
two_boxes_for_ar1 = True
limit_boxes = True # Whether or not you want to limit the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are scaled as in the original implementation
coords = 'centroids' # Whether the box coordinates to be used as targets for the model should be in the 'centroids' or 'minmax' format, see documentation
normalize_coords = True

det_model_path = "./"

In [8]:
project_path = "C:\\Users\\kalya\\Downloads\\"
#project_path = "/content/drive/My Drive/DLCP/"

In [9]:
train_images_path = project_path + 'WIDER_train.zip'
test_images_path = project_path + 'WIDER_val.zip'

In [10]:
import zipfile
archive = zipfile.ZipFile(train_images_path, 'r')
archive.extractall()

In [11]:
archive = zipfile.ZipFile(test_images_path, 'r')
archive.extractall()

In [12]:
data = np.load('./wider_train_small.npy').item()

In [15]:
### Printed first element to check the above given information.

for key in data:
    print (key)
    print (data[key])
    break

52--Photographers/52_Photographers_photographertakingphoto_52_582.jpg
['WIDER_train/images/52--Photographers/52_Photographers_photographertakingphoto_52_582.jpg', 'WIDER_train/images/52--Photographers/52_Photographers_photographertakingphoto_52_582.jpg', [300, 300], [[21, 667, 9, 655], 1]]


 As we can see from the above output all the information mentioned above is there for all the images.

### Now, load the files `wider_trian.npy` and `wider_val.npy`

In [16]:
train_data = 'wider_train_small.npy'
test_data = 'wider_val_small.npy'

x = np.load(train_data)

### Now, call the imported model with the given parameters and freeze all the layers in the model with names not having ''`detection`'' word as prefix.

As we are not training the model from scratch, we are freezing all the above layers in the model having only last few layers while training to update their weights according to the problem in hand. This is called as **Transfer Learning**.

In [135]:
# build the keras model
# this model is not retrained, we are doing it from scratch 

K.clear_session()
model, model_layer, img_input, predictor_sizes = mn_model(image_size=(img_height, img_width, img_channels), 
                                                                      n_classes = n_classes,
                                                                      min_scale = None, 
                                                                      max_scale = None, 
                                                                      scales = scales, 
                                                                      aspect_ratios_global = None, 
                                                                      aspect_ratios_per_layer = aspect_ratios, 
                                                                      two_boxes_for_ar1= two_boxes_for_ar1, 
                                                                      limit_boxes=limit_boxes, 
                                                                      variances= variances, 
                                                                      coords=coords, 
                                                                      normalize_coords=normalize_coords)

====> Model Specific data
====> Height, Width, Channels : 512 512 3


#### Write code to freeze all the layers in the above model with names not having ''`detection`'' word as prefix.

In [139]:
import itertools

In [140]:
for (i,j) in zip(model.layers, model_layer): 
    if j[0:9]  != 'detection':
        i.trainable=False

In [141]:
for layer in model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x0000022F21F75550> False
<keras.layers.core.Lambda object at 0x0000022F287CFD68> False
<keras.layers.core.Lambda object at 0x0000022F287FF748> False
<keras.layers.core.Lambda object at 0x0000022F287E0358> False
<keras.layers.convolutional.Conv2D object at 0x0000022F287E0EF0> False
<keras.layers.normalization.BatchNormalization object at 0x0000022F287E8518> False
<keras.layers.core.Activation object at 0x0000022F21EB8748> False
<mn_model.DepthwiseConv2D object at 0x0000022F287E0D68> False
<keras.layers.normalization.BatchNormalization object at 0x0000022F287DBC50> False
<keras.layers.core.Activation object at 0x0000022F289D5DA0> False
<keras.layers.convolutional.Conv2D object at 0x0000022F28A35DD8> False
<keras.layers.normalization.BatchNormalization object at 0x0000022F28B05D30> False
<keras.layers.core.Activation object at 0x0000022F28ADAF28> False
<mn_model.DepthwiseConv2D object at 0x0000022F28B43B38> False
<keras.layers.normalization.

### After making the model ready for transfer learning, load the weights of the model given in file ''`mobilenet_1_0_224_tf.h5`''

In [137]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
lambda1 (Lambda)                (None, 512, 512, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda2 (Lambda)                (None, 512, 512, 3)  0           lambda1[0][0]                    
__________________________________________________________________________________________________
lambda3 (Lambda)                (None, 512, 512, 3)  0           lambda2[0][0]                    
__________________________________________________________________________________________________
conv1 (Con

__________________________________________________________________________________________________
conv_dw_13_relu (Activation)    (None, 16, 16, 1024) 0           conv_dw_13_bn[0][0]              
__________________________________________________________________________________________________
conv_pw_13 (Conv2D)             (None, 16, 16, 1024) 1048576     conv_dw_13_relu[0][0]            
__________________________________________________________________________________________________
conv_pw_13_bn (BatchNormalizati (None, 16, 16, 1024) 4096        conv_pw_13[0][0]                 
__________________________________________________________________________________________________
conv_pw_13_relu (Activation)    (None, 16, 16, 1024) 0           conv_pw_13_bn[0][0]              
__________________________________________________________________________________________________
detection_conv6_1 (Conv2D)      (None, 16, 16, 256)  262400      conv_pw_13_relu[0][0]            
__________

__________________________________________________________________________________________________
detection_conv9_2_mbox_loc_conv (None, 1, 1, 256)    0           detection_conv9_2_mbox_loc_conv_d
__________________________________________________________________________________________________
detection_conv4_3_norm_mbox_con (None, 32, 32, 8)    32          detection_conv4_3_norm_mbox_conf_
__________________________________________________________________________________________________
detection_fc7_mbox_conf_conv_pw (None, 16, 16, 12)   48          detection_fc7_mbox_conf_conv_pw_2
__________________________________________________________________________________________________
detection_conv6_2_mbox_conf_con (None, 8, 8, 12)     48          detection_conv6_2_mbox_conf_conv_
__________________________________________________________________________________________________
detection_conv7_2_mbox_conf_con (None, 4, 4, 12)     48          detection_conv7_2_mbox_conf_conv_
__________

In [142]:
model.compile('Adam',loss='mse')

In [143]:
model.load_weights('mobilenet_1_0_224_tf.h5',by_name=True)

#### Using the functions given in the model, we are trying to divide the dataset into train and validation samples. Run the below code.

In [144]:
batch_size = 32
ssd_box_encoder = SSDBoxEncoder(img_height=img_height,
                                img_width=img_width,
                                n_classes=n_classes, 
                                predictor_sizes=predictor_sizes,
                                min_scale=None,
                                max_scale=None,
                                scales=scales,
                                aspect_ratios_global=None,
                                aspect_ratios_per_layer=aspect_ratios,
                                two_boxes_for_ar1=two_boxes_for_ar1,
                                limit_boxes=limit_boxes,
                                variances=variances,
                                pos_iou_threshold=0.5,
                                neg_iou_threshold=0.2,
                                coords=coords,
                                normalize_coords=normalize_coords)

train_dataset = BatchGenerator(images_path=train_data, 
                include_classes='all', 
                box_output_format = ['class_id', 'xmin', 'xmax', 'ymin', 'ymax'])

print ("==>TRAINING DATA")
print ("==> Parsing XML files ...")

train_dataset.parse_xml(
                  annotations_path=train_data,
                  image_set_path='None',
                  image_set='None',
                  classes = class_names, 
                  exclude_truncated=False,
                  exclude_difficult=False,
                  ret=False, 
                  debug = False)
print("==>Parsing XML Finished.")

print ("==>Generate training batches...")
train_generator = train_dataset.generate(
                 batch_size=batch_size,
                 train=True,
                 ssd_box_encoder=ssd_box_encoder,
                 equalize=True,
                 brightness=(0.5,2,0.5),
                 flip=0.5,
                 translate=((0, 20), (0, 30), 0.5),
                 scale=(0.75, 1.2, 0.5),
                 crop=False,
                 #random_crop = (img_height,img_width,1,3), 
                 random_crop=False,
                 resize=(img_height, img_width),
                 #resize=False,
                 gray=False,
                 limit_boxes=True,
                 include_thresh=0.4,
                 diagnostics=False)

print ("==>Training batch generation complete")

n_train_samples = train_dataset.get_n_samples()

print ("==>Total number of training samples = {}".format(n_train_samples))

# Now repeat above steps for validation data 

print ("==>VALIDATION")

val_dataset = BatchGenerator(images_path=test_data, include_classes='all', 
                box_output_format = ['class_id', 'xmin', 'xmax', 'ymin', 'ymax'])

print ("==> Parsing XML files ...")


val_dataset.parse_xml(
                  annotations_path=test_data,
                  image_set_path='None',
                  image_set='None',
                  classes = class_names, 
                  exclude_truncated=False,
                  exclude_difficult=False,
                  ret=False, 
                  debug = False)


print("==>Parsing XML Finished.")


print ("==>Generate training batches...")
val_generator = val_dataset.generate(
                 batch_size=batch_size,
                 train=True,
                 ssd_box_encoder=ssd_box_encoder,
                 equalize=False,
                 brightness=False,
                 flip=False,
                 translate=False,
                 scale=False,
                 crop=False,
                 #random_crop = (img_height,img_width,1,3), 
                 random_crop=False, 
                 resize=(img_height, img_width), 
                 #resize=False, 
                 gray=False,
                 limit_boxes=True,
                 include_thresh=0.4,
                 diagnostics=False)


print ("==>Training batch generation complete")

n_val_samples = val_dataset.get_n_samples()

print ("==>Total number of validation samples = {}".format(n_val_samples))

==>TRAINING DATA
==> Parsing XML files ...
==>Parsing XML Finished.
==>Generate training batches...
==>Training batch generation complete
==>Total number of training samples = 128
==>VALIDATION
==> Parsing XML files ...
==>Parsing XML Finished.
==>Generate training batches...
==>Training batch generation complete
==>Total number of validation samples = 60
